In [ ]:
# Python3

In [ ]:
# Uncomment the following cell if running as a Binder notebook, in order to hide the Jupyter header
# content and controls, so giving the appearance of a standalone application. Note that if you run
# this cell manually in a regular notebook session then the Jupyter header content and controls
# will still be hidden. Run this cell again to toggle back:

In [ ]:
%%javascript
$('#header').toggle()

In [22]:
import ipywidgets as widgets
import re
import requests
from datetime import datetime
from ipyleaflet import FullScreenControl, LayerGroup, Map, Marker
from IPython.display import HTML
from urllib.parse import quote

In [23]:
# Layout styles
layout_main = {
    'width': '100%',
    'height': '100%',
}
layout_dropdown = {
    'width': '150px',
}
layout_button = {
    'width': '50px',
}

In [24]:
# Image widgets
image_header = widgets.Image(
    value=open('images/Minipedia_Header.png', 'rb').read(),
    format='png',
    layout=layout_main,
)

In [25]:
# Input widgets
input_text = widgets.Text(
    placeholder='Search for ...',
    layout=layout_main,
)

In [26]:
# Control widgets
dropdown_set_mode = widgets.Dropdown(
    value=0,
    options=[
        ('All outputs', 0),
        ('Description', 1),
        ('Extract', 2),
        ('Categories', 3),
        ('Geo', 4),
    ],
    layout=layout_dropdown,
)
button_toggle_output_open = widgets.Button(
    icon='list',
    tooltip='Toggle all output displays',
    layout=layout_button,
)
button_clear = widgets.Button(
    icon='refresh',
    tooltip='Clear input and output',
    layout=layout_button,
)
button_go = widgets.Button(
    icon='play',
    tooltip='Go!',
    layout=layout_button,
)

In [27]:
# Output widgets
output_message = widgets.Output()
output_des = widgets.Output()
output_ext = widgets.Output()
output_cat = widgets.Output()
output_geo = widgets.Output()

In [28]:
# Container widgets
accordion_des = widgets.Accordion(
    children=[output_des],
    layout=layout_main,
)
accordion_ext = widgets.Accordion(
    children=[output_ext],
    layout=layout_main,
)
accordion_cat = widgets.Accordion(
    children=[output_cat],
    layout=layout_main,
)
accordion_geo = widgets.Accordion(
    children=[output_geo],
    layout=layout_main,
)
accordion_des.set_title(0, 'Description')
accordion_ext.set_title(0, 'Extract')
accordion_cat.set_title(0, 'Categories')
accordion_geo.set_title(0, 'Geo')

children_header = [
    image_header,
    input_text,
]

children_control = [
    dropdown_set_mode,
    button_toggle_output_open,
    button_clear,
    button_go,
    output_message,
]

children_output = []
outputs = []

HBox_control = widgets.HBox(
    children=children_control,
)

VBox_main = widgets.VBox(
    layout = {
        'width': '90%',
        'min_height': '1500px',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)

In [29]:
@output_message.capture(clear_output=True, wait=True)
def set_output_message(html):
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_des.capture(clear_output=True, wait=True)
def set_output_des(html):
    accordion_des.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_ext.capture(clear_output=True, wait=True)
def set_output_ext(html):
    accordion_ext.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_cat.capture(clear_output=True, wait=True)
def set_output_cat(html):
    accordion_cat.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_geo.capture(clear_output=True, wait=True)
def set_output_geo(places):
    m = Map(
        center=(20.0, 0.0),
        zoom=2,
    )
    markers = []
    for place in places.keys():
        if  ('lat' in places[place].keys()) \
        and ('lon' in places[place].keys()):
            markers.append(Marker(
                location=(places[place]['lat'], places[place]['lon']),
                title=place,
                draggable=False,
            ))
    m.add_layer(LayerGroup(
        layers=tuple(markers),
    ))
#     m.add_layer(MarkerCluster(
#         markers=tuple(markers),
#     ))
    m.add_control(FullScreenControl())
    accordion_geo.selected_index = 0
    display(m)

In [30]:
def set_output(dummy):
    global children_output
    global outputs
    children_output = []
    outputs = []
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == 'Description'):
        children_output.append(accordion_des)
        outputs.append(output_des)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == 'Extract'):
        children_output.append(accordion_ext)
        outputs.append(output_ext)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == 'Categories'):
        children_output.append(accordion_cat)
        outputs.append(output_cat)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == 'Geo'):
        children_output.append(accordion_geo)
        outputs.append(output_geo)
    VBox_main.children = children_header + [HBox_control] + children_output

# -------------------------------------------------------------------------------------------------

def clear_output(dummy):
    for child in [accordion_des, accordion_ext, accordion_cat, accordion_geo]:
        child.selected_index = None
    for output in [output_des, output_ext, output_cat, output_geo]:
        output.clear_output()

# -------------------------------------------------------------------------------------------------

def clear_input_output(dummy):
    input_text.value=''
    output_message.clear_output()
    clear_output(None)

# -------------------------------------------------------------------------------------------------

def toggle_control(dummy):
    control_disabled = children_control[0].disabled
    for child in children_control:
        child.disabled = not control_disabled

# -------------------------------------------------------------------------------------------------

def toggle_output_open(dummy):
    output_open = any([(child.selected_index == 0) for child in children_output])
    for child in children_output:
        child.selected_index = None if output_open else 0

# -------------------------------------------------------------------------------------------------

def go(dummy):
    if (not input_text.value):
        set_output_message('ERROR: Enter text for analysis')
    else:

        toggle_control(None)
        set_output_message('Processing ...') # + '<i class="fa fa-gear fa-spin" style="font-size:18px"></i>')

        # -------------------------------------------------------------------------------------------------

        input_text_norm = input_text.value
        input_text_norm = input_text_norm[0].capitalize() + (input_text_norm[1:] if (len(input_text_norm) > 1) else '')
        input_text_norm = '_'.join(input_text_norm.split(' '))
        input_text_norm = input_text_norm.replace('|','-')
        input_text_norm = input_text_norm.encode('utf-8')
        input_text_norm = quote(input_text_norm)

        # -------------------------------------------------------------------------------------------------

        url_modes = []

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Description')):
            url_modes.append('description')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Extract')):
            url_modes.append('extracts')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Categories')):
            url_modes.append('categories')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Geo')):
            url_modes.append('coordinates')

        url_modes = '|'.join(url_modes)

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Categories')):
            url_modes += '&clshow=!hidden&cllimit=max'

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Geo')):
            url_modes += '&coprop=type|region|country'

        url = 'https://en.wikipedia.org'
        # url = 'https://simple.wikipedia.org'
        url += '/w/api.php'
        url += '?action=query&format=json&redirects&prop='
        url += url_modes
        url += '&titles='
        url += input_text_norm

        # -------------------------------------------------------------------------------------------------

        t1 = datetime.now()
        r = requests.get(url)
        t2 = datetime.now()

        deltat = t2 - t1
        deltat = round(deltat.seconds + (deltat.microseconds / 1e6), 6)

        # -------------------------------------------------------------------------------------------------

        toggle_control(None)
        clear_output(None)

        # -------------------------------------------------------------------------------------------------

        if  (r.ok) \
        and (r.json()) \
        and ('query' in r.json().keys()) \
        and ('pages' in r.json()['query'].keys()):

            set_output_message('Done in: ' + str(deltat) + ' s')

            pageid = list(r.json()['query']['pages'].keys())[0]
            title = r.json()['query']['pages'][pageid]['title']

            query_norm = None
            if 'normalized' in r.json()['query'].keys():
                query_norm = r.json()['query']['normalized'][0]['to']

            query_redirect = None
            if 'redirects' in r.json()['query'].keys():
                query_redirect = r.json()['query']['redirects'][0]['to']

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Description')) \
            and ('description' in r.json()['query']['pages'][pageid].keys()):
                set_output_des(r.json()['query']['pages'][pageid]['description'])

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Extract')) \
            and ('extract' in r.json()['query']['pages'][pageid].keys()):
                set_output_ext(r.json()['query']['pages'][pageid]['extract'])

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Categories')) \
            and ('categories' in r.json()['query']['pages'][pageid].keys()):
                set_output_cat('<br/>'.join(
                    cat
                    for cat in [
                        re.search(r'Category:(.*)', cat['title'])[1]
                        for cat in r.json()['query']['pages'][pageid]['categories']
                    ]))

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == 'Geo')) \
            and ('coordinates' in r.json()['query']['pages'][pageid].keys()):

                set_output_geo({title: {
                    'lat': r.json()['query']['pages'][pageid]['coordinates'][0]['lat'],
                    'lon': r.json()['query']['pages'][pageid]['coordinates'][0]['lon'],
                }})

                geo_type = None
                if 'type' in r.json()['query']['pages'][pageid]['coordinates'][0].keys():
                    geo_type = r.json()['query']['pages'][pageid]['coordinates'][0]['type']

                geo_region = None
                if 'region' in r.json()['query']['pages'][pageid]['coordinates'][0].keys():
                    geo_region = r.json()['query']['pages'][pageid]['coordinates'][0]['region']

                geo_country = None
                if 'country' in r.json()['query']['pages'][pageid]['coordinates'][0].keys():
                    geo_country = r.json()['query']['pages'][pageid]['coordinates'][0]['country']

        else:
            set_output_message('ERROR: No data received')
            clear_output(None)

In [31]:
dropdown_set_mode.observe(set_output)
button_toggle_output_open.on_click(toggle_output_open)
button_clear.on_click(clear_input_output)
button_go.on_click(go)

In [32]:
set_output(None)
clear_output(None)
display(VBox_main)